In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('course_lead_scoring.csv')

In [ ]:
df.head()

In [ ]:
df['industry'].mode()

In [ ]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [ ]:
categorical = list(df.dtypes[df.dtypes == 'object'].index)
categorical

In [ ]:
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']

In [ ]:
df[numerical]

In [ ]:
for c in categorical:
    df[c] = df[c].str.lower().str.replace(' ', '_')

In [ ]:
df.head().T

In [ ]:
df[numerical]=df[numerical].fillna(0)

In [ ]:
df[categorical]=df[categorical].fillna('No Value')

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [ ]:
len(df_train), len(df_val), len(df_test)

In [ ]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [ ]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [ ]:
del df_train['lead_score']
del df_val['lead_score']
del df_test['lead_score']

In [ ]:
df_full_train.isnull().sum()

In [ ]:
df_full_train.converted.value_counts(normalize=True)

In [ ]:
from IPython.display import display

In [ ]:
global_conversion_count = df['converted'].mean()

In [ ]:
global_conversion_count

In [ ]:
combined_list = numerical + categorical
for c in combined_list:
    print(c)
    df_group = df.groupby(c).converted.agg(['mean', 'count'])
    df_group['diff'] = df_group['mean'] - global_conversion_count
    df_group['risk'] = df_group['mean'] / global_conversion_count
    display(df_group)
    print()

In [ ]:
from sklearn.metrics import mutual_info_score

In [ ]:
def mutual_info_lead_score(series):
    return mutual_info_score(series, df_train.converted)

In [ ]:
mi = df_train[categorical].apply(mutual_info_lead_score)
mi.sort_values(ascending=False)

In [ ]:
corr = df[numerical].corrwith(df.lead_score).abs()
corr.sort_values(ascending=False)

In [ ]:
corr = df[numerical].corrwith(df.interaction_count).abs()
corr.sort_values(ascending=False)

In [ ]:
from sklearn.feature_extraction import DictVectorizer

In [ ]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.coef_[0].round(2)

In [ ]:
y_pred = model.predict_proba(X_val)[:, 1]

In [ ]:
convert_decision = (y_pred >= 0.5)

In [ ]:
(y_val == convert_decision).mean()

In [ ]:
df_pred = pd.DataFrame()
df_pred['probability'] = y_pred
df_pred['prediction'] = convert_decision.astype(int)
df_pred['actual'] = y_val

In [ ]:
df_pred['correct'] = df_pred.prediction == df_pred.actual

In [ ]:
df_pred.correct.mean()

In [ ]:
accuracy = accuracy_score(y_val, convert_decision)
accuracy

In [ ]:
dict(zip(dv.get_feature_names_out(), model.coef_[0].round(2)))

In [ ]:
categorical

In [ ]:
categorical_minus_industry = ['lead_source', 'employment_status', 'location']

In [ ]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical_minus_industry].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical_minus_industry].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [ ]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict_proba(X_val)[:, 1]

In [ ]:
convert_decision = (y_pred >= 0.5)

In [ ]:
df_pred = pd.DataFrame()
df_pred['probability'] = y_pred
df_pred['prediction'] = convert_decision.astype(int)
df_pred['actual'] = y_val

In [ ]:
df_pred['correct'] = df_pred.prediction == df_pred.actual

In [ ]:
df_pred.correct.mean()

In [ ]:
categorical_minus_empStatus = ['lead_source', 'industry', 'location']

In [ ]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical_minus_empStatus].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical_minus_empStatus].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [ ]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict_proba(X_val)[:, 1]

In [ ]:
convert_decision = (y_pred >= 0.5)

In [ ]:
(y_val == convert_decision).mean()

In [ ]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [ ]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)
model.coef_[0].round(2)
y_pred = model.predict_proba(X_val)[:, 1]
convert_decision = (y_pred >= 0.5)
(y_val == convert_decision).mean()

In [ ]:
model = LogisticRegression(solver='liblinear', C=0.01, max_iter=1000, random_state=42)
model.fit(X_train, y_train)
model.coef_[0].round(2)
y_pred = model.predict_proba(X_val)[:, 1]
convert_decision = (y_pred >= 0.5)
(y_val == convert_decision).mean()

In [ ]:
model = LogisticRegression(solver='liblinear', C=0.1, max_iter=1000, random_state=42)
model.fit(X_train, y_train)
model.coef_[0].round(2)
y_pred = model.predict_proba(X_val)[:, 1]
convert_decision = (y_pred >= 0.5)
(y_val == convert_decision).mean()

In [ ]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(X_train, y_train)
model.coef_[0].round(2)
y_pred = model.predict_proba(X_val)[:, 1]
convert_decision = (y_pred >= 0.5)
(y_val == convert_decision).mean()

In [ ]:
model = LogisticRegression(solver='liblinear', C=100, max_iter=1000, random_state=42)
model.fit(X_train, y_train)
model.coef_[0].round(2)
y_pred = model.predict_proba(X_val)[:, 1]
convert_decision = (y_pred >= 0.5)
(y_val == convert_decision).mean()